In [1]:
import numpy as np
import cv2

<h1 style = "text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d8/dfe/classcv_1_1VideoCapture.html">VideoCapture</a>
</h1>
<h2 style = "text-align: center">
is class for video capturing from video files, image sequences or cameras.
</h2>

In [3]:
cap = cv2.VideoCapture(0) # 0 is a id-number of video devices

while cap.isOpened() :
    #Read new frame
    ret, frame = cap.read()
    if ret == True :
        cv2.imshow('Frame', frame)
        
        if cv2.waitKey(33) & 0xFF == ord('q') : # Period control f - 1/T
            break
    else :
        break
    
cap.release()
cv2.destroyAllWindows()

In [4]:
vid = cv2.VideoCapture('./videos/Ant Tracking Hard.mp4')

while vid.isOpened() :
    ret, frame = vid.read()
    
    if ret :
        cv2.imshow('Video frame', frame)

        if cv2.waitKey(int(1000/24)) & 0xFF == ord('q') : # this line control the period between image frame
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()

<h1 style="text-align: center">
    Lucus-Kanade Sparse Optical Flow 
</h1> <br>
<h2 style="text-align: left">
    <a href="https://docs.opencv.org/4.5.3/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323"> calcOpticalFlowPyrLK()</a>
    is a Lucas-Kanade Spares Optical Flow built-in OpenCV function inside 
    <a href="https://docs.opencv.org/4.5.3/dc/d6b/group__video__track.html"> Object Tracking </a>
    module.
</h2>

In [5]:
def lucas_kanade_optical_flow(video_device) :

    cap = cv2.VideoCapture(video_device)
    # params for ShiTomasi corner detection
    feature_params = dict( maxCorners = 500,
                        qualityLevel = 0.03,
                        minDistance = 7,
                        blockSize = 25 )

    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (21,21),
                    maxLevel = 3,
                    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    #Create some random colors
    colors = np.random.randint(0, 255, (500, 3)) # 500 values 3 channel

    #Take first frame and find corner
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params) # Feature detection, Harris corner with Shi-Tomasi response function

    # Create a mask image for drawing overlay
    mask = np.zeros_like(old_frame)

    while cap.isOpened() :
        
        ret, frame = cap.read()

        if ret :
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            #calculate optical flow 
            p1, st, err = cv2.calcOpticalFlowPyrLK(
                old_gray, frame_gray, p0, None, **lk_params
            )

            # Select good points
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            # Traceline drawing
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel().astype(int)
                c, d = old.ravel().astype(int)
                mask = cv2.line(mask, (a, b), (c, d), colors[i].tolist(), 2)
                frame = cv2.circle(frame, (a,b), 5, colors[i].tolist(), -1)
            
            compare_img = cv2.hconcat([frame, mask])
            disp_img = cv2.add(frame, mask)
            cv2.imshow('compare', compare_img)
            cv2.imshow('frame', disp_img)

            key = cv2.waitKey(27) & 0xFF
            if key == 27 or key == ord('q') :
                break
            elif key == ord('c') : # clear mask
                mask = np.zeros_like(old_frame)
                p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
            else :
                #Update the previous frame and previous points
                old_gray = frame_gray.copy()
                p0 = good_new.reshape(-1, 1, 2)
        else :
            break

    cap.release()
    cv2.destroyAllWindows()

lucas_kanade_optical_flow(0)

<h1 style="text-align: center">
    Farnebäck Optical Flow
</h1> <br>
<h2 style="text-align: left">
    <a href="https://docs.opencv.org/4.5.3/dc/d6b/group__video__track.html#ga5d10ebbd59fe09c5f650289ec0ece5af"> calcOpticalFlowFarneback()</a>
    is a Farnebäck Dense Optical Flow built-in OpenCV function inside 
    <a href="https://docs.opencv.org/4.5.3/dc/d6b/group__video__track.html"> Object Tracking </a>
    module. <br>
    <a href="https://docs.opencv.org/4.5.3/d2/de8/group__core__array.html#gac5f92f48ec32cacf5275969c33ee837d">cartToPolar() </a>
    is a utility function for cartesian to polar conversion
</h2>

In [6]:
def farneback_dense_optical_flow(video_device) :
    cap = cv2.VideoCapture(video_device)

    ret, frame = cap.read()
    last_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame)
    hsv[:, :, 1] = 255

    while cap.isOpened() :
        ret, frame = cap.read()

        if ret :
            
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            flow = cv2.calcOpticalFlowFarneback(last_frame, frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0) # flow dx dy
            
            mag, ang = cv2.cartToPolar(flow[:, :, 0], flow[:, :, 1])
            hsv[:, :, 0] = ang*(180/np.pi/2)
            hsv[:, :, 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

            flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

            # thresh = cv2.inRange(hsv,(0, 0, 30), (20,255,255))
            # motion_segment = cv2.bitwise_and(frame, frame, mask=thresh )
            # cv2.imshow('thresh', motion_segment)
           
            last_frame = frame_gray.copy()

            cv2.imshow('frame', frame)
            cv2.imshow('flow', flow_rgb)
            key = cv2.waitKey(27) & 0xFF
            if key == 27 or key == ord('q') :
                break

        else :
            break
    
    cap.release()
    cv2.destroyAllWindows()
            

farneback_dense_optical_flow('./videos/Flow Visualization.mp4')

<h1 style="text-align: center">
    Optical Flow exercise
</h1>

<h2>แบบฝึกหัดที่ 10</h2>
<h4>วัตถุประสงค์ </h1>

- ทักษะการประยุกต์ใช้เทคนิค optical flow
<h4>โจทย์</h4>

- ให้นักศึกษาเขียน code ซอฟต์แวร์นำข้อมูลวิดีโอมาประมวลผลเพื่อติดตามการเคลื่อนไหวของคนหรือสิ่งของภายในภาพ (Crowd heatmapping)
- ให้นักศึกษากำหนด ROI (พื้นที่ที่สนใจของภาพ, พื้นที่ในกรอบสี่เหลี่ยมของภาพ) เพื่อป้องกันการตรวจจับเสาและสภาพแวดล้อมที่ไม่เกี่ยวข้องภายนอก
- ในการส่งงานให้นักศึกษาคอมเมนต์ code ที่ตนเองเขียนและอัพโหลดไปยัง github ของตนเองแล้วนำลิงก์ดังกล่าวไปโพสต์ส่งใน googleclassroom
- video ต่าง ๆ ที่ถูกในในตัวอย่างด้านบนอยู่ภายใน directory <a href="https://github.com/jbinteam/010723305/tree/main/videos">videos</a>
- ชุดข้อมูลวิดีโอ <a href = "https://github.com/jbinteam/010723305/blob/main/videos/grandcentral.mp4?raw=true">grandcentral.mp4</a><br>
- ผลลัพธ์ที่คาดหวัง <a href ="https://youtu.be/UoXAaafHeQY" >Youtube Video</a> 


In [2]:
## coding here :D
def lucas_kanade_optical_flow(video_device) :   #ฟังก์ชัน lucas_kanade_optical_flow

    cap = cv2.VideoCapture(video_device)        #เรียกใช้งานวิดิโอ
  
    feature_params = dict( maxCorners = 500,    #เก็บค่าต่างๆใน dictionary เพื่อใช้สำหรับคำสั่ง goodFeaturesToTrack
                        qualityLevel = 0.01,    
                        minDistance = 7,
                        blockSize = 25 )

    lk_params = dict( winSize  = (21,21),       #เก็บค่าต่างๆใน dictionary เพื่อใช้สำหรับคำสั่ง calcOpticalFlowPyrLK
                    maxLevel = 3,
                    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    colors = np.random.randint(0, 255, (500, 3)) #สุ่มค่า 0 - 255 มาจำนวน 500 ค่า 3 แชลแนล

    ret, old_frame = cap.read()                 #อ่านวิดิโอ
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)              #แปลงภาพให้เป็น Grayscale
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params) #ตรวจจับฟีเจอร์ด้วยคำสั่ง goodFeaturesToTrack

    mask = np.zeros_like(old_frame)             #สร้าง mask ที่มีขนาดเท่ากับ old_frame

    points = np.array([[180,30],[25,480],[720,480],[550,30]],np.int32)  #กำหนดจุดมุมของสี่เหลี่ยมที่ต้องการตรวจจับฟีเจอร์
    roi = np.zeros(old_frame.shape[:2],np.uint8)                        #สร้างภาพสีดำขนาด 480 x 720
    cv2.drawContours(roi,[points],-1,255,-1,cv2.LINE_AA)                #วาดพท้นที่ที่สนใจด้วยสีขาว

    while cap.isOpened() :                      #วนลูปเมื่อวิดิโอทำงานอยู่
        ret, frame = cap.read()                 #อ่าน frame
        if ret :                                #ตรวจสอบค่า ret ว่ามีค่าเป็น true หรือไม่?
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)        #แปลงภาพให้เป็น Grayscale  

            p1, st, err = cv2.calcOpticalFlowPyrLK(                     #คำนวณ OpticalFlow ด้วยคำสั่ง calcOpticalFlowPyrLK 
                old_gray, frame_gray, p0, None, **lk_params 
            )

            good_new = p1[st == 1]              #หา p1 ที่มีค่า st เท่ากับ 1
            good_old = p0[st == 1]              #หา p0 ที่มีค่า st เท่ากับ 1

            for i, (new, old) in enumerate(zip(good_new, good_old)):     #วนลูปตามจำนวนค่า good_new และ good_old รวมกัน
                a, b = new.ravel().astype(int)  #เก็บค่าตำแหน่งภาพใหม่ใน a และ b
                c, d = old.ravel().astype(int)  #เก็บค่าตำแหน่งภาพเก่าใน c และ d
                if b >= frame.shape[0]:         #ตรวจสอบไม่ให้ค่า b มีค่ามากกว่าหรือเท่ากับ 480
                    b = frame.shape[0]-1        
                if a >= frame.shape[1]:         #ตรวจสอบไม่ให้ค่า a มีค่ามากกว่าหรือเท่ากับ 720
                    a = frame.shape[1]-1
                if roi[b,a] == 255:             #ตรวจสอบค่า roi ที่ตำแหน่ง b,a เท่ากับ 255 หรือไม่?
                    mask = cv2.line(mask, (a, b), (c, d), colors[i].tolist(), 1)    #วาดเส้นจาก a,b ไปหา c,d
                    frame = cv2.circle(frame, (a,b), 3, colors[i].tolist(), -1)     #วาดวงกลมที่ a,b ขนาดเท่ากับ 3 px
            
            compare_img = cv2.hconcat([frame, mask])        #นำภาพ frame กับ mask มาต่อกัน
            disp_img = cv2.add(frame, mask)                 #นำภาพ frame กับ mask มาประกบกัน
            cv2.drawContours(disp_img,[points],-1,(0,0,255),1,cv2.LINE_AA)          #วาดเส้นขอบสีแดง
            cv2.imshow('compare', compare_img)              #แสดงภาพเปรียบเทียบระหว่าง frame กับ mask
            cv2.imshow('frame', disp_img)                   #แสดงภาพผลลัพธ์

            key = cv2.waitKey(int(1000/30)) & 0xFF          #อ่านค่า waitKey เท่ากับ 1000 / 30 frame มีค่าเท่ากับ 33           
            if key == 27 or key == ord('q') :               #ตรวจสอบการกดปุ่ม esc หรือ q
                break                                       #ออกจากลูป while
            elif key == ord('c') or len(good_new) < 495 :   #ตรวจสอบการกดปุ่ม c หรือ good_new มีค่าน้อยกว่า 495
                p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)  #ตรวจจับฟีเจอร์ใหม่
            else :                                          #ถ้าไม่มีการกดปุ่มให้ภาพและตำแหน่งใหม่กลายเป็นภาพและตำแหน่งเก่า
                old_gray = frame_gray.copy()
                p0 = good_new.reshape(-1, 1, 2)
        else :                                              #ถ้า ret เป็น false ทำการออกจากลูป while
            break

    cap.release()                 #ส่งค่าคืน
    cv2.destroyAllWindows()       #ปิดหน้าจอแสดงผลทั้งหมด

lucas_kanade_optical_flow('./videos/grandcentral.mp4') #ส่งวิดิโอ grandcentral.mp4 เข้าไปยังฟังก์ชัน lucas_kanade_optical_flow